### Machine Learning with Big Data
## Part 2: Convolutional Neural Networks

# 0.2 - Train, Test & Split

**Group:** MLBD_Deventer 4   
**Students:** Mathijs Pattipeilohy (512391), Emin Fikret (511275)

In [1]:
import os
import numpy as np
import pandas as pd
import sklearn as sk
import tensorflow as tf

from joblib import load, dump

2025-10-24 12:38:59.199518: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-24 12:38:59.245624: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-24 12:38:59.433041: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-24 12:39:00.600502: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
ARTIFACTS_DIR = "../artifacts"
INPUT_FILE  = os.path.join(ARTIFACTS_DIR, "images_df.joblib")
OUTPUT_FILE = os.path.join(ARTIFACTS_DIR, "images_tuple.joblib")

In [3]:
images_df = load(INPUT_FILE)
print(f"Loaded DataFrame with {len(images_df)} images")

Loaded DataFrame with 1286 images


In [4]:
X = np.stack(images_df["Image"].values).astype("float32")
y = images_df["Label"].values

print("Initial array shape:", X.shape)

Initial array shape: (1286, 128, 128, 4)


In [5]:
le = sk.preprocessing.LabelEncoder()
y_int = le.fit_transform(y)
y = tf.keras.utils.to_categorical(y_int)

classes = list(le.classes_)
num_classes = len(classes)

print(f"Classes ({num_classes}):", classes)

Classes (12): ['az', 'dobro', 'estb', 'frbtb', 'ize', 'jatb', 'ljudie', 'nasb', 'sa', 'sta', 'ukb', 'vede']


In [6]:
# we could have used this as well: 
# https://www.tensorflow.org/tutorials/load_data/images

X_train, X_temp, y_train, y_temp = sk.model_selection.train_test_split(
    X, y, 
    test_size=0.3, 
    random_state=42, 
    stratify=y_int
)

y_temp_int = y_temp.argmax(axis=1)
X_val, X_test, y_val, y_test = sk.model_selection.train_test_split(
    X_temp, y_temp,
    test_size=0.33,
    random_state=42,
    stratify=y_temp_int
)

print("Train:", X_train.shape, y_train.shape) # 70%
print("Val:  ", X_val.shape,   y_val.shape) # 20%
print("Test: ", X_test.shape,  y_test.shape) # 10%

Train: (900, 128, 128, 4) (900, 12)
Val:   (258, 128, 128, 4) (258, 12)
Test:  (128, 128, 128, 4) (128, 12)


In [7]:
dump(
    (X_train, X_val, X_test, y_train, y_val, y_test, classes),
    OUTPUT_FILE,
    compress=3
)
print(f"Saved dataset splits to {OUTPUT_FILE}")

Saved dataset splits to ../artifacts/images_tuple.joblib
